In [ ]:
try:
  # Colab only
  %tensorflow_version 2.x
except Exception:
  pass

#import necessary libraries.
import tensorflow as tf
import numpy as np
layer = tf.keras.layers

print('check tensorflow version : ', tf.__version__)

In [ ]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

print("The shape of train dataset : ", x_train.shape)
print("The shape of test dataset : ", x_test.shape)

In [ ]:
train_ds = tf.data.Dataset.from_tensor_slices((x_train,y_train)).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((x_test,y_test)).batch(32)

# Customize model.fit

In [ ]:
# 각 train step(iteration)의 결과를 저장핳기 위한 metric 객체 정의 
mean_loss = tf.keras.metrics.Mean(name='loss')
mean_acc = tf.keras.metrics.SparseCategoricalAccuracy(name='accuracy')

In [ ]:
# loss function을 정의하여 커스터마이징
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy()
def custom_loss(target_y, predicted_y):
  return loss_fn(target_y, predicted_y)

In [ ]:
# tf.keras.Model의 train_step과 test_step 메서드를 오버라이딩한다.
class CustomModel(tf.keras.Model):
    def train_step(self, data):
        x, y = data

        with tf.GradientTape() as tape:
            y_pred = self(x, training=True)  # Forward pass
            # Compute our own loss
            loss = custom_loss(y, y_pred)

        # Compute gradients
        trainable_vars = self.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)

        # Update weights
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))

        # Compute our own metrics
        mean_loss.update_state(loss)
        mean_acc.update_state(y, y_pred)
        return {m.name: m.result() for m in self.metrics}
    
    def test_step(self, data):
        # Unpack the data
        x, y = data
        # Compute predictions
        y_pred = self(x, training=False)
        # Updates the metrics tracking the loss
        loss = custom_loss(y, y_pred)
        # Update the metrics.
        mean_loss.update_state(loss)
        mean_acc.update_state(y, y_pred)
        # Return a dict mapping metric names to current value.
        # Note that it will include the loss (tracked in self.metrics).
        return {m.name: m.result() for m in self.metrics}
    
    @property
    def metrics(self):
        # We list our `Metric` objects here so that `reset_states()` can be
        # called automatically at the start of each epoch
        # or at the start of `evaluate()`.
        # If you don't implement this property, you have to call
        # `reset_states()` yourself at the time of your choosing.
        return [mean_loss, mean_acc]

In [ ]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(64, activation='relu'),
  tf.keras.layers.Dense(64, activation='relu'),
  tf.keras.layers.Dense(10, activation='softmax')
])
model.build([None, 28, 28])

In [ ]:
custom_model = CustomModel(model.input, model.output)
optimizer = tf.keras.optimizers.Adam()
# We don't passs a loss or metrics here.
custom_model.compile(optimizer)

In [ ]:
custom_model.fit(x_train, y_train, epochs=5, validation_split=0.2)
custom_model.evaluate(x_test, y_test)

# Loop training

model.fit 메서드는 편리한 인터페이스를 제공하지만 유연한 커스터마이징이 어렵다. Loop training 방식은 Loop 구조 내에서 tf.GradientTape을 사용해 학습을 정교하게 제어하는 방식이다.

In [ ]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(64, activation='relu'),
  tf.keras.layers.Dense(64, activation='relu'),
  tf.keras.layers.Dense(10, activation='softmax')
])

In [ ]:
# 각 train step(iteration)의 결과를 저장핳기 위한 metric 객체 정의 
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

In [ ]:
# loss function을 정의하여 커스터마이징
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy()
def custom_loss(target_y, predicted_y):
  return loss_fn(target_y, predicted_y)
optimizer = tf.keras.optimizers.Adam()

In [ ]:
# mini-batch에 대한 train_step과 test_step을 정의
@tf.function
def train_step(images, labels):
  with tf.GradientTape() as tape:
    # training=True is only needed if there are layers with different
    # behavior during training versus inference (e.g. Dropout).
    predictions = model(images, training=True)
    loss = custom_loss(labels, predictions)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

  train_loss(loss)
  train_accuracy(labels, predictions)
    
@tf.function
def test_step(images, labels):
  # training=False is only needed if there are layers with different
  # behavior during training versus inference (e.g. Dropout).
  predictions = model(images, training=False)
  t_loss = custom_loss(labels, predictions)

  test_loss(t_loss)
  test_accuracy(labels, predictions)

In [ ]:
EPOCHS = 5

for epoch in range(EPOCHS):
    # Reset the metrics at the start of the next epoch
    train_loss.reset_states()
    train_accuracy.reset_states()
    test_loss.reset_states()
    test_accuracy.reset_states()

    for images, labels in train_ds:
        train_step(images, labels)

    for test_images, test_labels in test_ds:
        test_step(test_images, test_labels)

    print(
        f'Epoch {epoch + 1}, '
        f'Loss: {train_loss.result()}, '
        f'Accuracy: {train_accuracy.result() * 100}, '
        f'Test Loss: {test_loss.result()}, '
        f'Test Accuracy: {test_accuracy.result() * 100}')